In [1]:
from bokeh.plotting import figure, save, show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColumnDataSource, Range1d, HoverTool, LogColorMapper
from bokeh.resources import CDN
from bokeh.palettes import Viridis6 as palette
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
import pysal as ps
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

/home/powerchen/.local/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
PATH = r"mapdata/TOWN_MOI_1071031.shp"
villages_shp = gpd.read_file(PATH, encoding='utf-8') #全台灣村里界圖
villages_shp = villages_shp.to_crs({'init' :'epsg:4326'})

DriverError: mapdata/TOWN_MOI_1071031.shp: No such file or directory

In [ ]:
villages_shp.head()

In [ ]:
def cartesian(x): 
    return np.vstack(np.array([np.array(np.meshgrid(*i)).T.reshape(-1,8) for i in x.values]))

In [ ]:
villages_shp = gpd.GeoDataFrame(cartesian(villages_shp),columns=villages_shp.columns)

In [ ]:
villages_shp.head(6)

In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [ ]:
villages_shp['x'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
villages_shp['y'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [ ]:
#villages_shp[['x', 'y']].head(6)
villages_shp.head(6)

In [ ]:
#villages_shp_source = GeoJSONDataSource(geojson=villages_shp.to_json())
source = ColumnDataSource(data=dict(
    x=villages_shp.x,
    y=villages_shp.y,
    cty=villages_shp.COUNTYNAME + ", " + villages_shp.TOWNNAME
))

TOOLTIPS = [
    ("Latitude", "$x"),
    ("Longitude", "$y"),
    ("County", "@cty"),   
]
color_mapper = LogColorMapper(palette=palette)
#HoverTool.mode="vline"

In [ ]:
# Initialize our plot figure
p = figure(title="Taiwan Map", output_backend="webgl", plot_width=800, plot_height=600,
          tools="pan,wheel_zoom,box_zoom,reset")
p.axis.visible=False
p.add_tools(HoverTool(
    tooltips=TOOLTIPS,

    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode='vline'
))

In [ ]:
# Add the lines to the map from our GeoJSONDataSource -object
#p.multi_line('x', 'y', source=source, color='black', line_width=2)
p.patches('x', 'y', source=source,
          fill_color={'field': 'x', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

In [ ]:
output_file("taiwanmap.html")
#output_notebook()

In [ ]:
#show(p)
save(p, title="taiwanmap")